In [3]:
!pip install --upgrade openai --quiet

In [4]:
with open('key', 'r') as file:
    openai_key = file.read()

In [5]:
import openai
client = openai.OpenAI(api_key=openai_key)

In [6]:
from openai.types import Model, ModelDeleted

In [7]:
models = client.models.list()

In [8]:
type(models)

openai.pagination.SyncPage[Model]

In [15]:
for k in models:
    if "gpt" in k.id:
        print(k.id)

gpt-4-vision-preview
gpt-3.5-turbo-0613
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-3.5-turbo-instruct
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-1106
gpt-4
gpt-4-1106-preview
gpt-4-0613
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-16k


In [17]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [18]:
text = f"""
step 1: extract keypoints and descriptors from all images
step 2: find matches between all images
step 3: select a reference image
step 4: begin adding the other images together with the reference image by
  first finding a transformation between them
  and then blending the images together"""

In [19]:
prompt = f"""
I'm coding in python and using the OpenCV library. Give me a python script to build panoramas using the following algorithm:
```{text}```
"""

response = get_completion(prompt)
print(response)

Here is a simple script that follows the steps you've outlined using OpenCV and Python:

```python
import cv2
import numpy as np

def stitch_images(images):
    # Step 1: Detect keypoints and extract local invariant descriptors
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints = []
    descriptors = []
    for image in images:
        keypoint, descriptor = sift.detectAndCompute(image, None)
        keypoints.append(keypoint)
        descriptors.append(descriptor)

    # Step 2: Match descriptors between images
    matcher = cv2.DescriptorMatcher_create("BruteForce")
    raw_matches = matcher.knnMatch(descriptors[0], descriptors[1], 2)

    # Lowe's ratio test
    good_points = []
    for m, n in raw_matches:
        if m.distance < 0.75 * n.distance:
            good_points.append(m)

    # Step 3: Select a reference image
    # In this case, we'll just use the first image as the reference
    ref_image = images[0]

    # Step 4: Find a transformation between the reference image 